#### Serialization and Composiotion

디스크에서 Prompt template 을 가져오는 방법에 대해 학습합니다.

Prompt 를 어디에 저장해두고, 다른 누구나 프롬프르를 가져다 쓸 수 있도록 하고 싶을 떄 유용합니다.

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt


In [22]:
chat = ChatOpenAI(
    temperature=0.1,        # 창의성 (0 ~ 2)
    model='gpt-3.5-turbo',  # 사용 모델 지정 (Default : gpt-3.5-turbo)
    streaming=True,         # Streaming ON
    callbacks=[StreamingStdOutCallbackHandler()]
)

#### 💡 JSON, YAML 형태의 프롬프트 파일 불러오기

In [19]:
# prompt = load_prompt('./prompt.yaml')
prompt = load_prompt('./prompt.json')
prompt.format(country='한국')

'한국의 수도는 어디야?'

#### 💡 PipelinePromptTemplate

많은 Prompt들의 Memory 등을 다 모으는 방법에 대해 알아보겠습니다.

많은 Prompt들을 하나로 합칠 수 있도록 해줍니다.

In [24]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    당신은 역할 수행 보조자입니다.
    그리고 당신은 {character}를 사칭하고 있습니다.
    """
)

example = PromptTemplate.from_template(
    """
    이것은 당신이 말하는 방식의 예시입니다:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    지금 시작하세요!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final, 
    pipeline_prompts=prompts
)

full_prompt.format(
    character="강아지",
    example_question="너는 어디 사니 ?",
    example_answer="크르를ㅇ! 쾅르릉ㅇ!! 왈왈!!",
    question="너가 가장 좋아하는 음식은 뭐야?"
)

chain = full_prompt | chat

chain.invoke({
    "character":"강아지",
    "example_question":"너는 어디 사니 ?",
    "example_answer":"크르를ㅇ! 쾅르릉ㅇ!! 왈왈!!",
    "question":"너가 가장 좋아하는 음식은 뭐야?"
})

왈왈!! 고기!! 왈왈!!

AIMessage(content='왈왈!! 고기!! 왈왈!!', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-f7f95387-3329-4c9b-9722-c0ee93bb34cf-0')